### Utils

In [85]:
import base64
import os
import json
from datetime import datetime
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage
import requests
import re
import json
from typing import Dict, Any
from dotenv import load_dotenv
import time

load_dotenv()
# GitHub API endpoint
owner = "VarjuAkos"
repo = "HealthTrack-Pro"
api_url = "https://api.github.com/repos/VarjuAkos/HealthTrack-Pro/issues"

token = os.getenv("GITHUB_TOKEN")


def load_markdown_to_str(file_path):
	with open(file_path, 'r', encoding='utf-8') as md_file:
		markdown_content = md_file.read()
	return markdown_content

def load_latest_sprint_status(base_path):
    """
    Find the latest sprint directory and load the project-sprint-status.md file.
    
    Args:
    base_path (str): Path to the directory containing sprint folders.
    
    Returns:
    str: Content of the project-sprint-status.md file from the latest sprint.
    """
    try:
        # List all directories in the base path
        directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
        
        # Filter and sort sprint directories
        sprint_dirs = sorted([d for d in directories if d.startswith('sprint') and d[6:].isdigit()],
                             key=lambda x: int(x[6:]),
                             reverse=True)
        
        if not sprint_dirs:
            return "No sprint directories found."
        
        # Get the latest sprint directory
        latest_sprint = sprint_dirs[0]
        sprint_path = os.path.join(base_path, latest_sprint)
        
        # Look for project-sprint-status.md in the latest sprint directory
        status_file = os.path.join(sprint_path, 'project-sprint-status.md')
        
        if os.path.exists(status_file):
            with open(status_file, 'r', encoding='utf-8') as f:
                return f.read()
        else:
            return f"project-sprint-status.md not found in {latest_sprint}."
    
    except Exception as e:
        return f"An error occurred: {str(e)}"
    
    
def load_latest_sprint_backlog(base_path):
    """
    Find the latest sprint directory and load the project-sprint-backlog.json file.
    
    Args:
    base_path (str): Path to the directory containing sprint folders.
    
    Returns:
    dict: Content of the project-sprint-backlog.json file from the latest sprint.
    """
    try:
        # List all directories in the base path
        directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
        
        # Filter and sort sprint directories
        sprint_dirs = sorted([d for d in directories if d.startswith('sprint') and d[6:].isdigit()],
                             key=lambda x: int(x[6:]),
                             reverse=True)
        
        if not sprint_dirs:
            return {"error": "No sprint directories found."}
        
        # Get the latest sprint directory
        latest_sprint = sprint_dirs[0]
        sprint_path = os.path.join(base_path, latest_sprint)
        
        # Look for project-sprint-backlog.json in the latest sprint directory
        backlog_file = os.path.join(sprint_path, 'project-sprint-backlog.json')
        
        if os.path.exists(backlog_file):
            with open(backlog_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        else:
            return {"error": f"project-sprint-backlog.json not found in {latest_sprint}."}
    
    except Exception as e:
        return {"error": f"An error occurred: {str(e)}"}
    
def export_transcript(state, folder_path):
    # Create the folder if it doesn't exist
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    id = len(files) + 1
    transcript = state["transcript"]
    
    filename = state["meeting_type"].replace(" ","_") + str(id) + ".txt"
    
    os.makedirs(os.path.join(folder_path), exist_ok=True)
    
    
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)
    
    # Write the string to a text file
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(transcript)

def export_state(state, folder_path, filename):
    # Create the folder if it doesn't exist
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    id = len(files) + 1
    
    filename = filename+str(id)+".json"
    
    os.makedirs(folder_path, exist_ok=True)
    
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)
    
    # Write the state dict to a JSON file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(state, f, indent=4)

def load_txt_to_str(file_path):
    with open(file_path, 'r', encoding='utf-8') as txt_file:
        text_content = txt_file.read()
    return text_content

def load_from_json(file_path):
    """
    Load data from a JSON file.
    
    Args:
    file_path (str): Path to the JSON file.
    
    Returns:
    dict: A dictionary containing the loaded JSON data.
    """
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return {}
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file {file_path}")
        return {}
    
def format_mermaid(input_string):
    # Step 1: Extract the mermaid code block
    mermaid_pattern = r'```mermaid\n([\s\S]*?)\n```'
    match = re.search(mermaid_pattern, input_string)
    
    if match:
        mermaid_code = match.group(1)
    else:
        # If no mermaid code block is found, return the original string
        return input_string
    
    # Step 2: Replace escaped newlines with actual newlines
    formatted_string = mermaid_code.replace(r'\n', '\n')
    
    # Step 3: Strip any leading/trailing whitespace
    formatted_string = formatted_string.strip()
    
    return formatted_string

def render_mermaid_diagram(diagram_code: str) -> str:
    # First, format the diagram code
    formatted_code = format_mermaid(diagram_code)
    
    # Encode the formatted Mermaid code
    encoded_diagram = base64.b64encode(formatted_code.encode('utf-8')).decode('utf-8')
    
    # Make a request to the Mermaid rendering service
    url = f"https://mermaid.ink/img/{encoded_diagram}"
    response = requests.get(url)
    
    if response.status_code == 200:
        # Return the URL of the rendered image
        return url
    else:
        # If rendering failed, return the formatted Mermaid code
        return f"```mermaid\n{formatted_code}\n```"


def export_meeting_history(state, output_file='meeting_history.json'):
    """
    Export the meeting history to a JSON file.
    
    Args:
    state (dict): The state dictionary containing the meeting history.
    output_file (str): The name of the output file. Defaults to 'meeting_history.json'.
    
    Returns:
    None
    """
    meeting_history = state.get("meeting_history", [])
    
    # Ensure meeting_history is a list
    if not isinstance(meeting_history, list):
        meeting_history = [meeting_history]
    
    try:
        with open(output_file, 'w') as file:
            json.dump(meeting_history, file, indent=2)
        print(f"Meeting history exported successfully to {output_file}")
    except IOError:
        print(f"Error: Unable to write to file {output_file}")

    return state  # Return the state to maintain consistency with your workflow


def manage_sprint_folders(state, project_folder):
    """
    Manages sprint folders based on the meeting type.
    Creates a new sprint folder if necessary and adds required files.
    
    :param state: The current state dictionary
    :param project_folder: Path to the project folder
    :return: Updated state with new sprint information
    """
    if "planning" in state.get("meeting_type", "").lower() or "plan" in state.get("meeting_type", "").lower():
        # List all directories in the project folder
        directories = [d for d in os.listdir(project_folder) if os.path.isdir(os.path.join(project_folder, d))]
        
        # Filter and find the highest sprint number
        sprint_numbers = [int(re.findall(r'\d+', d)[0]) for d in directories if d.startswith("sprint") and re.findall(r'\d+', d)]
        
        if sprint_numbers:
            new_sprint_number = max(sprint_numbers) + 1
        else:
            new_sprint_number = 1
        
        # Create new sprint folder
        new_sprint_folder = os.path.join(project_folder, f"sprint{new_sprint_number}")
        os.makedirs(new_sprint_folder, exist_ok=True)
        
        # Create project_sprint_status.md
        status_file_path = os.path.join(new_sprint_folder, "project_sprint_status.md")
        with open(status_file_path, 'w') as status_file:
            status_file.write(f"# Sprint {new_sprint_number} Status\n\nStatus details will be updated here.")
        
        # Create project_sprint_backlog.json
        backlog_file_path = os.path.join(new_sprint_folder, "project_sprint_backlog.json")
        initial_backlog = {}
        with open(backlog_file_path, 'w') as backlog_file:
            json.dump(initial_backlog, backlog_file, indent=2)
        
        # Update state with new sprint information
        state["current_sprint_number"] = new_sprint_number
        state["current_sprint_folder"] = new_sprint_folder
        state["sprint_status_file"] = status_file_path
        state["sprint_backlog_file"] = backlog_file_path
        
        print(f"Created new sprint folder: {new_sprint_folder}")
    else:
        print("Meeting type does not indicate a planning session. No new sprint folder created.")
    
    return state


def load_json(file_path: str) -> Dict[str, Any]:
    """
    Reads a JSON file and returns its contents as a dictionary.

    :param file_path: The path to the JSON file to be read
    :return: A dictionary containing the data from the JSON file
    :raises FileNotFoundError: If the specified file does not exist
    :raises json.JSONDecodeError: If the file is not valid JSON
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        print(f"Successfully loaded JSON from {file_path}")
        return data
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: The file {file_path} is not valid JSON. Error: {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred while reading {file_path}: {str(e)}")
        raise


def get_latest_sprint_folder(project_folder: str) -> str:
    """
    Scans the project folder for sprint folders and returns the name of the latest sprint folder.

    :param project_folder: Path to the project folder (e.g., 'project1/')
    :return: Name of the latest sprint folder (e.g., 'sprint5'), or None if no sprint folders are found
    """
    # List all items in the project folder
    items = os.listdir(project_folder)

    # Filter for sprint folders and extract their numbers
    sprint_folders = []
    for item in items:
        if os.path.isdir(os.path.join(project_folder, item)):
            match = re.match(r'sprint(\d+)', item, re.IGNORECASE)
            if match:
                sprint_number = int(match.group(1))
                sprint_folders.append((item, sprint_number))

    # Sort sprint folders by number (descending) and return the latest
    if sprint_folders:
        latest_sprint = max(sprint_folders, key=lambda x: x[1])
        print(f"Latest sprint folder found: {latest_sprint[0]}")
        return latest_sprint[0]
    else:
        print("No sprint folders found.")
        return None


def export_markdown(content, file_path):
    """
    Export content to a Markdown file.
    
    Args:
    content (str): The content to be written to the file.
    file_path (str): The path where the file should be saved.
    """
    try:
        folder_path = os.path.dirname(file_path)
        os.makedirs(folder_path, exist_ok=True)
        
        # Count existing files in the folder
        existing_files = len([f for f in os.listdir(folder_path) if f.endswith('.md')])
        
        # Create new filename with incremented number
        file_name = os.path.basename(file_path)
        name, ext = os.path.splitext(file_name)
        new_file_name = f"{name}_{existing_files + 1}{ext}"
        new_file_path = os.path.join(folder_path, new_file_name)
        
        with open(new_file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"Successfully exported Markdown to {new_file_path}")
    except Exception as e:
        print(f"Error exporting Markdown: {str(e)}")

def export_json(content, file_path):
    """
    Export content to a JSON file.
    
    Args:
    content (dict): The content to be written to the file.
    file_path (str): The path where the file should be saved.
    """
    try:
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(content, f, indent=2)
        print(f"Successfully exported JSON to {file_path}")
    except Exception as e:
        print(f"Error exporting JSON: {str(e)}")



def export_backlog(content, file_path):
    """
    Export content to a JSON file.
    Args:
    content (str or dict): The content to be written to the file. Can be a JSON string or a dictionary.
    file_path (str): The path where the file should be saved.
    """
    try:
        # If content is a string, parse it into a dictionary
        if isinstance(content, str):
            # Replace escaped newlines with actual newlines
            content = content.replace("\\n", "\n")
            # Parse the JSON string
            content_dict = json.loads(content)
        elif isinstance(content, dict):
            content_dict = content
        else:
            raise ValueError("Content must be either a JSON string or a dictionary")

        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(file_path), exist_ok=True)

        # Write the dictionary to a JSON file
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(content_dict, f, indent=2)

        print(f"Successfully exported JSON to {file_path}")

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {str(e)}")
        print("Content causing the error:")
        print(content)
    except Exception as e:
        print(f"Error exporting JSON: {str(e)}")

        
def load_latest_json(folder_path: str) -> Dict[str, Any]:
    """
    Reads the latest JSON file from a given folder and returns its contents as a dictionary.

    :param folder_path: The path to the folder containing JSON files
    :return: A dictionary containing the data from the latest JSON file
    :raises FileNotFoundError: If no JSON files are found in the specified folder
    :raises json.JSONDecodeError: If the latest file is not valid JSON
    """
    try:
        # Get all JSON files in the folder
        json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
        
        if not json_files:
            raise FileNotFoundError(f"No JSON files found in {folder_path}")
        
        # Find the latest JSON file
        latest_file = max(json_files, key=lambda f: os.path.getmtime(os.path.join(folder_path, f)))
        latest_file_path = os.path.join(folder_path, latest_file)
        
        # Read and parse the latest JSON file
        with open(latest_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        print(f"Successfully loaded latest JSON from {latest_file_path}")
        return data
    
    except FileNotFoundError:
        print(f"Error: No JSON files found in {folder_path}")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: The file {latest_file_path} is not valid JSON. Error: {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred while reading from {folder_path}: {str(e)}")
        raise
    
def load_second_latest_json(folder_path: str) -> Dict[str, Any]:
    """
    Reads the second latest JSON file from a given folder and returns its contents as a dictionary.

    :param folder_path: The path to the folder containing JSON files
    :return: A dictionary containing the data from the second latest JSON file
    :raises FileNotFoundError: If fewer than two JSON files are found in the specified folder
    :raises json.JSONDecodeError: If the second latest file is not valid JSON
    """
    try:
        # Get all JSON files in the folder
        json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
        
        if len(json_files) < 2:
            raise FileNotFoundError(f"Fewer than two JSON files found in {folder_path}")
        
        # Sort JSON files by modification time, newest first
        sorted_files = sorted(json_files, key=lambda f: os.path.getmtime(os.path.join(folder_path, f)), reverse=True)
        
        # Get the second latest file
        second_latest_file = sorted_files[1]
        second_latest_file_path = os.path.join(folder_path, second_latest_file)
        
        # Read and parse the second latest JSON file
        with open(second_latest_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        print(f"Successfully loaded second latest JSON from {second_latest_file_path}")
        return data
    
    except FileNotFoundError:
        print(f"Error: Fewer than two JSON files found in {folder_path}")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: The file {second_latest_file_path} is not valid JSON. Error: {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred while reading from {folder_path}: {str(e)}")
        raise 
def load_employee_profiles(file_path):
    """
    Load employee profiles from a JSON file.
    
    Args:
    file_path (str): Path to the JSON file containing employee profiles.
    
    Returns:
    dict: A dictionary of employee profiles.
    """
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return {}
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file {file_path}")
        return {}
    
import os
import json
from typing import Dict, Any

def load_latest_note_taker_json(folder_path: str) -> Dict[str, Any]:
    """
    Reads the latest JSON file containing 'note_taker' in its name from the specified folder.
    
    :param folder_path: The path to the folder containing the JSON files
    :return: A dictionary containing the data from the latest note_taker JSON file
    :raises FileNotFoundError: If no matching file is found in the specified folder
    :raises json.JSONDecodeError: If the file is not valid JSON
    """
    try:
        # Get all files in the folder that contain 'note_taker' and end with '.json'
        note_taker_files = [f for f in os.listdir(folder_path) if 'note_taker' in f and f.endswith('.json')]
        
        if not note_taker_files:
            raise FileNotFoundError(f"No 'note_taker' JSON files found in {folder_path}")
        
        # Sort files by modification time (most recent first)
        latest_file = max(note_taker_files, key=lambda f: os.path.getmtime(os.path.join(folder_path, f)))
        
        file_path = os.path.join(folder_path, latest_file)
        
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        print(f"Successfully loaded latest note_taker JSON from {file_path}")
        return data
    
    except FileNotFoundError:
        print(f"Error: No 'note_taker' JSON files found in {folder_path}")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: The file {file_path} is not valid JSON. Error: {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred while reading from {folder_path}: {str(e)}")
        raise


def create_github_issue(backlog_item):
    # Create the issue body
    issue_body = f"""
    Description: {backlog_item['description']}
    Status: {backlog_item['status']}
    Priority: {backlog_item['priority']}
    Estimated Effort: {backlog_item['estimated_effort']}
    Assigned To: {backlog_item['assigned_to']}
    Tags: {', '.join(backlog_item['tags'])}
    ID: {backlog_item['id']}
    """

    # Prepare the data for the API request
    data = {
        "title": backlog_item['title'],
        "body": issue_body,
        "labels": backlog_item['tags']
    }

    # Set up the headers for authentication
    headers = {
        "Authorization": f"token {token}",
        "Accept": "application/vnd.github.v3+json"
    }

    # Send the POST request to create the issue
    response = requests.post(api_url.format(owner=owner, repo=repo), headers=headers, data=json.dumps(data))

    # Check the response
    if response.status_code == 201:
        print(f"Issue created successfully: {backlog_item['title']}")
        print(f"Issue URL: {response.json()['html_url']}")
        return response.json()['html_url']
    else:
        print(f"Failed to create issue for: {backlog_item['title']}")
        print(f"Status code: {response.status_code}")
        print(f"Response: {response.text}")
        return None

def upload_backlog_to_github(backlog_json_path):
    # Load the backlog JSON
    with open(backlog_json_path, 'r') as file:
        backlog_data = json.load(file)

    created_issues = []

    # Iterate through each backlog item and create an issue
    for item in backlog_data['backlog']:
        issue_url = create_github_issue(item)
        if issue_url:
            created_issues.append({
                'id': item['id'],
                'title': item['title'],
                'url': issue_url
            })
        
        # Add a small delay to avoid hitting GitHub's rate limit
        time.sleep(1)

    # Optionally, you can save the created issues to a file
    with open('created_issues.json', 'w') as file:
        json.dump(created_issues, file, indent=2)

    print(f"Created {len(created_issues)} issues on GitHub.")

### Prompts

In [28]:
WORKFLOW = """
The workflow consist of several chains in sequence to get the desired outcome. 

The desired outcomes are these 4 type of document.
1. A structured Sprint planning Summary
2. A structured backlog suggestion with Epics/stories/tasks
3. JSON formatted backlog items that could be used to push to github
4. Personal summaries based on responsibilities 

The workflow conation these steps: 
1. Goals, tasks, purpose. From meeting note and basic info (company, project state, requirements, employees)
2. Identify valuable insights that is not contained in the previous steps.
3. Task breakdown suggestion Epics/story/tasks +
4. Backlog suggestion based on task breakdown
5. Backlog in JSON format +
6. Sprint planning summary draft
7. Sprint planning diagram Gannt chart
8. Sprint Summary compile +
9. Personal summary diagram
10. Personal summary compile +

I have marked those steps where the output of that given chain will be an actually document that the user will get.

Use this as a context so you can get the broad picture and see where your task is located in the landscape of steps.
You don't need to follow all these step just the one which us outlined.
"""


In [86]:
GOALS_TASK_PURPOSE_IDENTIFIER_PROMPT = """
You are the Scrum Master in a company. To accelerate your work and to get the best possible results you organize your work into a pipeline of steps.
Here is some information about the workflow you are working on:
<workflow>
{WORKFLOW}
</workflow>
Here you will find information about the company where you are giving advice as a Scrum Master: \n <company_data> \n {company_data} \n </company_data> \n
Here you will find information about the employees and their detailed profile: \n <employee_profiles> \n {employee_profiles} \n </employee_profiles> \n
The company currently working on this project: \n <project_general> \n {project_general} \n </project_general> \n
Here you will find the requirements that needs to be fufilled: \n <project_requirements> \n {project_requirements} \n </project_requirements> \n

Here you will find information about the project status overall: \n <project_state> \n {project_state} \n </project_state> \n
Here you will find the past meetings that happened. \n <meeting_history> \n {meeting_history} \n </meeting_history> \n

All this information that you have above is your context so you can perform your task.
But the main source of information is the meeting note that you will find below.
<meeting_note>
{meeting_note}
</meeting_note>
As said you are a Scrum Master helping for a company.
Now you are currently in step 1.
Step 1: Identify Goals, tasks, purpose.
Note that this step will be crucial and will determine the next steps' outcome so be precise and cautious.
"""

REFINED_INSIGHTS_PROMPT = """
You are the Scrum Master in a company. To accelerate your work and to get the best possible results you organize your work into a pipeline of steps.
Here is some information about the workflow you are working on:
\n <workflow> \n {WORKFLOW} \n </workflow> \n

Here you will find information about the company where you are giving advice as a Scrum Master: \n <company_data> \n {company_data} \n </company_data> \n
Here you will find information about the employees and their detailed profile: \n <employee_profiles> \n {employee_profiles} \n </employee_profiles> \n
The company currently working on this project: \n <project_general> \n {project_general} \n </project_general> \n
Here you will find the requirements that needs to be fulfilled: \n <project_requirements> \n {project_requirements} \n </project_requirements> \n

All this information that you have above is your context so you can perform your task.
But the main source of information is the meeting note that you will find below.
\n <meeting_note> \n {meeting_note} \n </meeting_note> \n

As said you are a Scrum Master helping for a company.
Now you are currently in step 2.

Here is the output from the previous step:
\n <previous step> \n {goals_task_purpose_output} \n </previous step> \n

Step 2: Identify valuable insights that are not contained in the previous steps. This step is needed in case you left out something valuable.
Note it is always better to contain more information than to leave out something. Since your output will be processed and compressed later.
Answer with all the infromation you collected with this and with the previous step.
"""

TASK_BREAKDOWN_PROMPT = """
You are the Scrum Master in a company. To accelerate your work and to get the best possible results you organize your work into a pipeline of steps.
Here is some information about the workflow you are working on:
<workflow>
{WORKFLOW}
</workflow>
Here you will find information about the employees and their detailed profile: \n <employee_profiles> \n {employee_profiles} \n </employee_profiles> \n
The company currently working on this project: \n <project_general> \n {project_general} \n </project_general> \n
Here you will find the requirements that needs to be fulfilled: \n <project_requirements> \n {project_requirements} \n </project_requirements> \n

All this information that you have above is your context so you can perform your task.

But the main source of information is the meeting note AND the output from the previous step that you will find below.
\n <meeting_note> \n {meeting_note} \n </meeting_note> \n

Here is the output from the previous step:
<refined_insights_output>
{refined_insights_output}
</refined_insights_output>

As said you are a Scrum Master helping for a company.
Now you are currently in step 3.
Step 3: Create a complete analysis of the tasks and make a Task breakdown suggestion. Use Epics/Stories/Tasks this will be backbone of the project backlog for this sprint.
"""

BACKLOG_SUGGESTION_PROMPT = """
You are the Scrum Master in a company. To accelerate your work and to get the best possible results you organize your work into a pipeline of steps.
Here is some information about the workflow you are working on:
<workflow>
{WORKFLOW}
</workflow>
Here you will find information about the company where you are giving advice as a Scrum Master: \n <company_data> \n {company_data} \n </company_data> \n
The company currently working on this project: \n <project_general> \n {project_general} \n </project_general> \n
Here you will find the requirements that needs to be fulfilled: \n <project_requirements> \n {project_requirements} \n </project_requirements> \n
Here you will find information about the project status overall: \n <project_state> \n {project_state} \n </project_state> \n
All this information that you have above is your context so you can perform your task.

The main source of information is the meeting note and the output from the task breakdown step that you will find below.
\n <meeting_note> \n {meeting_note} \n </meeting_note> \n

Here is the output from the task breakdown step:
<task_breakdown_output>
{task_breakdown_output}
</task_breakdown_output>

Here you will find information about the employees and their detailed profile: \n <employee_profiles> \n {employee_profiles} \n </employee_profiles> \n

As said y Scrum Master helping for a company.
Now you are currently in step 4.
Step 4: Create a backlog suggestion based on the recent task breakdown. Assign the tasks to employees based on their responsibilities.
"""

JSON_BACKLOG_PROMPT = """
You are the Scrum Master in a company. To accelerate your work and to get the best possible results you organize your work into a pipeline of steps.
Here is some information about the workflow you are working on:
<workflow>
{WORKFLOW}
</workflow>

Here is the output from backlog suggestion step:
<backlog_suggestion_output>
{backlog_suggestion_output}
</backlog_suggestion_output>

As said you are a Scrum Master helping for a company.
Now you are currently in step 5.
Step 5: Generate the complete backlog in JSON format. Answer only with the JSON formatted backlog. Here is an example:
{{
  "backlog": [
    {{
      "id": "US001",
      "title": "Manually log daily activities",
      "description": "As a user, I want to manually log my daily activities to track my health progress.",
      "status": "Not Started",
      "priority": "High",
      "estimated_effort": "5 story points",
      "assigned_to": "Emily",
      "tags": [
        "feature",
        "frontend",
        "activity-tracking"
      ],
      "dependencies": []
    }},
...
}}
"""

SPRINT_PLANNING_DRAFT_PROMPT = """
You are the Scrum Master in a company. To accelerate your work and to get the best possible results you organize your work into a pipeline of steps.
Here is some information about the workflow you are working on:
<workflow>
{WORKFLOW}
</workflow>
Here you will find information about the company where you are giving advice as a Scrum Master: \n <company_data> \n {company_data} \n </company_data> \n
Here you will find information about the employees and their detailed profile: \n <employee_profiles> \n {employee_profiles} \n </employee_profiles> \n
The company currently working on this project: \n <project_general> \n {project_general} \n </project_general> \n
Here you will find the requirements that needs to be fulfilled: \n <project_requirements> \n {project_requirements} \n </project_requirements> \n

Here you will find information about the project status overall: \n <project_state> \n {project_state} \n </project_state> \n
Here you will find the past meetings that happened. \n <meeting_history> \n {meeting_history} \n </meeting_history> \n

All this information that you have above is your context so you can perform your task.

Here is the meeting note:
\n <meeting_note> \n {meeting_note} \n </meeting_note> \n

Here is the output from the task breakdown step:
<task_breakdown_output>
{task_breakdown_output}
</task_breakdown_output>

As said you are a Scrum Master helping for a company.
Now you are currently in step 6.
Step 6: Create a Sprint planning summary draft. Format your output like in this template:
{sprint_planning_template}
"""

SPRINT_PLANNING_DIAGRAM_PROMPT = """
You are the Scrum Master in a company. To accelerate your work and to get the best possible results you organize your work into a pipeline of steps.
Here is some information about the workflow you are working on:
<workflow>
{WORKFLOW}
</workflow>
Here you will find information about the employees and their detailed profile: \n <employee_profiles> \n {employee_profiles} \n </employee_profiles> \n

Here is the output from the task breakdown step:
<task_breakdown_output>
{task_breakdown_output}
</task_breakdown_output>

As said you are a Scrum Master helping for a company.
Now you are currently in step 7.
Step 7: Generate a diagram which represents the schedule of this sprint accurately. Use mermaid diagrams syntax for this. Create a Gantt chart that includes all tasks, their dependencies, and assigned team members.
"""

SPRINT_SUMMARY_COMPILE_PROMPT = """
You are the Scrum Master in a company. To accelerate your work and to get the best possible results you organize your work into a pipeline of steps.
Here is some information about the workflow you are working on:
<workflow>
{WORKFLOW}
</workflow>

Here is the output from the sprint planning draft step:
<sprint_planning_draft>
{sprint_planning_draft}
</sprint_planning_draft>

Here is the diagram from the sprint planning diagram step:
<sprint_planning_diagram>
{sprint_planning_diagram}
</sprint_planning_diagram>

Use this template to compile the final sprint planning summary:
\n <sprint_summary_compile_template> \n {sprint_planning_template} \n </sprint_summary_compile_template> \n

As said you are a Scrum Master helping for a company.
Now you are currently in step 8.
Step 8: Compile the final version of the Sprint planning summary. Use the provided template.
"""

PERSONAL_SUMMARY_DIAGRAM_PROMPT = """
You are the Scrum Master in a company. To accelerate your work and to get the best possible results you organize your work into a pipeline of steps.
Here is some information about the workflow you are working on:
####Workflow####
{WORKFLOW}
####Workflow####
Here you will find the meeting note: \n <meeting_note> \n {meeting_note} \n </meeting_note> \n
Here you will find the final version of the sprint planning summary: \n <sprint_summary_compile> \n {sprint_summary_compile} \n </sprint_summary_compile> \n


Here you will find information about the employe and their detailed profile: \n <employee_profiles> \n {employee_profiles} \n </employee_profiles> \n
Here you will find the backlog that will be worked on in this sprint: \n <backlog> \n {backlog_suggestion_output} \n </backlog> \n
All this information that you have above is your context so you can perform your task.

As said you are a Scrum Master helping for a company.
Now you are currently in step 9.
Step 9: Generate diagram for each employee based on their tasks and their responsibilities. 
Use mermaid diagrams syntax to create a personalized task flow or mind map for each team member, showing their assigned tasks, dependencies, and key responsibilities for the sprint.
"""

PERSONAL_SUMMARY_COMPILE_PROMPT = """
You are the Scrum Master in a company. To accelerate your work and to get the best possible results you organize your work into a pipeline of steps.
Here is some information about the workflow you are working on:
<Workflow>
{WORKFLOW}
</Workflow>
Here you will find the meeting note: \n <meeting_note> \n {meeting_note} \n </meeting_note> \n
Here you will find the final version of the sprint planning summary: \n <sprint_summary_compile> \n {sprint_summary_compile} \n </sprint_summary_compile> \n


Here you will find information about the employe and their detailed profile: \n <employee_profiles> \n {employee_profiles} \n </employee_profiles> \n
Here you will find the backlog that will be worked on in this sprint: \n <backlog> \n {backlog_suggestion_output} \n </backlog> \n
All this information that you have above is your context so you can perform your task.

As said you are a Scrum Master helping for a company.
Now you are currently in step 10.



Step 10: Compile a personalized summary for each employee. Add the diagrams generated in step 9. Use this template for each employee:
<personal_summary_template>
{personal_summary_template}
</personal_summary_template>
Make sure to customize this summary for each team member based on their specific roles, responsibilities, and assigned tasks. If there is missing or unknown information for any section, you may leave it out or indicate that it's not applicable.
"""


### State

In [30]:
from abc import ABC, abstractmethod
from typing import Dict, List, TypedDict

from typing import TypedDict, Dict, List

class SprintPlanningState(TypedDict):
    # General information
    company_data: str
    project_general: str
    project_requirements: str
    employee_profiles: str
    employee_profiles_json: Dict[str, Any]
    # Variable information
    project_state: str
    meeting_history : Dict[str, Any]
    # Note taker
    meeting_note: str

    # Goals and insights
    goals_task_purpose_output: str
    refined_insights_output: str
    
    # Task and backlog information
    task_breakdown_output: str
    backlog_suggestion_output: str
    json_backlog: Dict[str, Any]
    
    # Sprint planning and summary
    sprint_planning_draft: str
    sprint_planning_diagram: str
    sprint_summary_compile: str
    
    # Personal summaries
    personal_summary_diagrams: Dict[str, str]
    personal_reports: Dict[str, str]
    
    # Templates

    sprint_planning_template: str
    personal_summary_template: str
    
class Graph(ABC):
    @abstractmethod
    def create_graph(self):
        pass
    
    @abstractmethod
    def run_graph(self, project_folder):
        pass

### Node functions

In [80]:
import json
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage
from langchain_anthropic import ChatAnthropic
from  dotenv import load_dotenv
import os

load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
#model = ChatAnthropic(model="claude-3-haiku-20240307", anthropic_api_key=anthropic_api_key)
model = ChatAnthropic(model="claude-3-5-sonnet-20240620", anthropic_api_key=anthropic_api_key, max_tokens= 8192)

def goals_task_purpose_identifier_node(state: SprintPlanningState) -> SprintPlanningState:
    formatted_prompt = GOALS_TASK_PURPOSE_IDENTIFIER_PROMPT.format(
        WORKFLOW=WORKFLOW,
        company_data=state["company_data"],
        employee_profiles=state["employee_profiles"],
        project_general=state["project_general"],
        project_requirements=state["project_requirements"],
        project_state=state["project_state"],
        meeting_history=state["meeting_history"],
        meeting_note=state["meeting_note"]
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Identify goals, tasks, and purpose for this sprint.")
    ]
    response = model.invoke(messages)
    state["goals_task_purpose_output"] = response.content
    return state

def refined_insights_node(state: SprintPlanningState) -> SprintPlanningState:
    formatted_prompt = REFINED_INSIGHTS_PROMPT.format(
        WORKFLOW=WORKFLOW,
        company_data=state["company_data"],
        employee_profiles=state["employee_profiles"],
        project_general=state["project_general"],
        project_requirements=state["project_requirements"],
        project_state=state["project_state"],
        meeting_history=state["meeting_history"],
        meeting_note=state["meeting_note"],
        goals_task_purpose_output=state["goals_task_purpose_output"]
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Identify valuable insights not contained in the previous steps.")
    ]
    response = model.invoke(messages)
    state["refined_insights_output"] = response.content
    return state

def task_breakdown_node(state: SprintPlanningState) -> SprintPlanningState:
    formatted_prompt = TASK_BREAKDOWN_PROMPT.format(
        WORKFLOW=WORKFLOW,
        employee_profiles=state["employee_profiles"],
        project_general=state["project_general"],
        project_requirements=state["project_requirements"],
        meeting_note=state["meeting_note"],
        refined_insights_output=state["refined_insights_output"]
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Create a complete analysis of tasks and make a task breakdown suggestion.")
    ]
    response = model.invoke(messages)
    state["task_breakdown_output"] = response.content
    return state

def backlog_suggestion_node(state: SprintPlanningState) -> SprintPlanningState:
    formatted_prompt = BACKLOG_SUGGESTION_PROMPT.format(
        WORKFLOW=WORKFLOW,
        company_data=state["company_data"],
        project_general=state["project_general"],
        project_requirements=state["project_requirements"],
        project_state=state["project_state"],
        meeting_note=state["meeting_note"],
        task_breakdown_output=state["task_breakdown_output"],
        employee_profiles=state["employee_profiles"]
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Create a backlog suggestion based on the task breakdown.")
    ]
    response = model.invoke(messages)
    state["backlog_suggestion_output"] = response.content
    return state

def json_backlog_node(state: SprintPlanningState) -> SprintPlanningState:
    formatted_prompt = JSON_BACKLOG_PROMPT.format(
        WORKFLOW=WORKFLOW,
        backlog_suggestion_output=state["backlog_suggestion_output"]
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Generate the complete backlog in JSON format.")
    ]
    response = model.invoke(messages)
    state["json_backlog"] = response.content
    return state

def sprint_planning_draft_node(state: SprintPlanningState) -> SprintPlanningState:
    formatted_prompt = SPRINT_PLANNING_DRAFT_PROMPT.format(
        WORKFLOW=WORKFLOW,
        company_data=state["company_data"],
        employee_profiles=state["employee_profiles"],
        project_general=state["project_general"],
        project_requirements=state["project_requirements"],
        project_state=state["project_state"],
        meeting_history=state["meeting_history"],
        meeting_note=state["meeting_note"],
        task_breakdown_output=state["task_breakdown_output"],
        sprint_planning_template=state["sprint_planning_template"]
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Create a Sprint planning summary draft.")
    ]
    response = model.invoke(messages)
    state["sprint_planning_draft"] = response.content
    return state

def sprint_planning_diagram_node(state: SprintPlanningState) -> SprintPlanningState:
    formatted_prompt = SPRINT_PLANNING_DIAGRAM_PROMPT.format(
        WORKFLOW=WORKFLOW,
        employee_profiles=state["employee_profiles"],
        task_breakdown_output=state["task_breakdown_output"]
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Generate a diagram representing the schedule of this sprint.")
    ]
    response = model.invoke(messages)
    state["sprint_planning_diagram"] = render_mermaid_diagram(format_mermaid(response.content))
    return state

def sprint_summary_compile_node(state: SprintPlanningState) -> SprintPlanningState:
    formatted_prompt = SPRINT_SUMMARY_COMPILE_PROMPT.format(
        WORKFLOW=WORKFLOW,
        sprint_planning_draft=state["sprint_planning_draft"],
        sprint_planning_diagram=state["sprint_planning_diagram"],
        sprint_planning_template=state["sprint_planning_template"]
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Compile the final version of the Sprint planning summary.")
    ]
    response = model.invoke(messages)
    state["sprint_summary_compile"] = response.content
    return state

def personal_summary_diagram_node(state: SprintPlanningState) -> SprintPlanningState:
    state["personal_summary_diagrams"] = {}
    for employee, profile in state["employee_profiles_json"].items():
        formatted_prompt = PERSONAL_SUMMARY_DIAGRAM_PROMPT.format(
            WORKFLOW=WORKFLOW,
            meeting_note=state["meeting_note"],
            sprint_summary_compile=state["sprint_summary_compile"],
            employee_profiles=profile,
            backlog_suggestion_output=state["backlog_suggestion_output"]
        )
        messages = [
            SystemMessage(content=formatted_prompt),
            HumanMessage(content=f"Generate a diagram for {employee} based on their tasks and responsibilities.")
        ]
        response = model.invoke(messages)
        state["personal_summary_diagrams"][employee] = render_mermaid_diagram(format_mermaid(response.content))
    return state

def personal_summary_compile_node(state: SprintPlanningState) -> SprintPlanningState:
    state["personal_reports"] = {}
    for employee, profile in state["employee_profiles_json"].items():
        formatted_prompt = PERSONAL_SUMMARY_COMPILE_PROMPT.format(
            WORKFLOW=WORKFLOW,
            meeting_note=state["meeting_note"],
            sprint_summary_compile=state["sprint_summary_compile"],
            employee_profiles=profile,
            backlog_suggestion_output=state["backlog_suggestion_output"],
            personal_summary_template=state["personal_summary_template"]
        )
        messages = [
            SystemMessage(content=formatted_prompt),
            HumanMessage(content=f"Compile a personalized summary for {employee}.")
        ]
        response = model.invoke(messages)
        state["personal_reports"][employee] = response.content
    return state

### Graph

In [72]:
from langgraph.graph import StateGraph, END


class SprintPlanning(Graph):
    def __init__(self):
        self.workflow = None

    def create_graph(self):
        workflow = StateGraph(SprintPlanningState)

        # Add nodes
        workflow.add_node("goals_task_purpose_identifier_node", goals_task_purpose_identifier_node)
        workflow.add_node("refined_insights_node", refined_insights_node)
        workflow.add_node("task_breakdown_node", task_breakdown_node)
        workflow.add_node("backlog_suggestion_node", backlog_suggestion_node)
        workflow.add_node("json_backlog_node", json_backlog_node)
        workflow.add_node("sprint_planning_draft_node", sprint_planning_draft_node)
        workflow.add_node("sprint_planning_diagram_node", sprint_planning_diagram_node)
        workflow.add_node("sprint_summary_compile_node", sprint_summary_compile_node)
        workflow.add_node("personal_summary_diagram_node", personal_summary_diagram_node)
        workflow.add_node("personal_summary_compile_node", personal_summary_compile_node)

        # Define edges
        workflow.add_edge("goals_task_purpose_identifier_node", "refined_insights_node")
        workflow.add_edge("refined_insights_node", "task_breakdown_node")
        workflow.add_edge("task_breakdown_node", "backlog_suggestion_node")
        workflow.add_edge("backlog_suggestion_node", "json_backlog_node")
        workflow.add_edge("json_backlog_node", "sprint_planning_draft_node")
        workflow.add_edge("sprint_planning_draft_node", "sprint_planning_diagram_node")
        workflow.add_edge("sprint_planning_diagram_node", "sprint_summary_compile_node")
        workflow.add_edge("sprint_summary_compile_node", "personal_summary_diagram_node")
        workflow.add_edge("personal_summary_diagram_node", "personal_summary_compile_node")
        workflow.add_edge("personal_summary_compile_node", END)

        workflow.set_entry_point("goals_task_purpose_identifier_node")
        self.workflow = workflow.compile()

    def run_graph(self, project_folder) -> SprintPlanningState:
        if self.workflow is None:
            raise ValueError("Graph has not been created. Call create_graph() first.")

        state = init_state(project_folder)
        state = self.workflow.invoke(state)

        return state

    def export_files(self, state: SprintPlanningState, project_folder: str):
        #state
        export_state(state=state, folder_path=os.path.join(project_folder, "state-logs/"), filename="sprint_planning")
        #reports
        for employee, report in state["personal_reports"].items():
            export_markdown(report, os.path.join(project_folder, "reports", f"sprint_planning_{employee}_report.md"))
        #summary
        export_markdown(state["sprint_summary_compile"], os.path.join(project_folder, "reports", f"sprint_planning_summary.md"))
        #backlog
        export_backlog(state["json_backlog"], os.path.join(project_folder, "backlog", f"sprint_planning_backlog.json"))
        #backlog suggestion
        export_markdown(state["backlog_suggestion_output"], os.path.join(project_folder, "backlog", f"sprint_planning_backlog_suggestion.md"))


def init_state(project_folder = "../../../data_/project1/"):

    note_taker_state = load_latest_note_taker_json(os.path.join(project_folder, "state-logs"))


    state = SprintPlanningState(
        company_data=load_markdown_to_str(file_path=os.path.join(project_folder, "company-general.md")),
        project_general=load_markdown_to_str(file_path=os.path.join(project_folder, "project-general.md")),
        project_requirements= load_markdown_to_str(file_path=os.path.join(project_folder, "project-requirements.md")),
        employee_profiles=load_markdown_to_str(file_path=os.path.join(project_folder, "employee-profiles.md")),
        employee_profiles_json=load_json(file_path=os.path.join(project_folder, "employee-profiles.json")),

        project_state=load_markdown_to_str(file_path=os.path.join(project_folder, "project-state.md")),
        meeting_history=load_json(file_path=os.path.join(project_folder, "meeting-history.json")),
        meeting_note=note_taker_state["note_final"],

        goals_task_purpose_output="",
        refined_insights_output="",
        task_breakdown_output="",
        backlog_suggestion_output="",
        json_backlog={},
        sprint_planning_draft="",
        sprint_planning_diagram="",
        sprint_summary_compile="",
        personal_summary_diagrams={},
        personal_reports={},

        sprint_planning_template=load_markdown_to_str(file_path=os.path.join(project_folder, "template", "project-sprint-state-template.md")),
        personal_summary_template=load_markdown_to_str(file_path=os.path.join(project_folder, "template", "personalized-meeting-summary-template.md"))
    )
    return state


### Testing

In [ ]:
project_folder = "../../../data_/project1/"
state = init_state(project_folder)
goals_task_purpose_identifier_node(state)
refined_insights_node(state)
task_breakdown_node(state)
backlog_suggestion_node(state)
json_backlog_node(state)
sprint_planning_draft_node(state)
sprint_planning_diagram_node(state)
sprint_summary_compile_node(state)
personal_summary_diagram_node(state)
personal_summary_compile_node(state)

In [ ]:
graph = SprintPlanning()
graph.create_graph()
state = graph.run_graph(project_folder)
state

In [44]:
project_folder = "../../../data_/project1/"
state = init_state(project_folder)
goals_task_purpose_identifier_node(state)

Successfully loaded latest note_taker JSON from ../../../data_/project1/state-logs/note_taker2.json
Successfully loaded JSON from ../../../data_/project1/employee-profiles.json
Successfully loaded JSON from ../../../data_/project1/meeting-history.json


{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [45]:
refined_insights_node(state)

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [46]:
task_breakdown_node(state)

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [47]:
backlog_suggestion_node(state)

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [49]:
state

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [51]:
json_backlog_node(state)

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [52]:
sprint_planning_draft_node(state)


{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [53]:
sprint_planning_diagram_node(state)

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [54]:
sprint_summary_compile_node(state)

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [55]:
personal_summary_diagram_node(state)

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [56]:
personal_summary_compile_node(state)

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

TODO:

structured output (reports, summary)
Summary very bad - note will be used instead of this
Report elejerol ki kell szedni certainly sort


####Daily meeting/sprint retrospective/Sprint planning#####


In [73]:
graph = SprintPlanning()
graph.create_graph()
graph.export_files(state, project_folder)


Successfully exported JSON to ../../../data_/project1/backlog/sprint_planning_backlog.json


In [78]:
render_mermaid_diagram(format_mermaid(state["personal_summary_diagrams"]["Alex Rodriguez"]))

'https://mermaid.ink/img/Z3JhcGggVEQKICAgIEFbQWxleCBSb2RyaWd1ZXo8YnI+U2VuaW9yIEZ1bGwtU3RhY2sgRGV2ZWxvcGVyXQogICAgCiAgICBBIC0tPiBCW01haW4gUmVzcG9uc2liaWxpdGllc10KICAgIEEgLS0+IENbU3ByaW50IFRhc2tzXQogICAgCiAgICBCIC0tPiBCMVtMZWFkIFRlY2huaWNhbCBEZWNpc2lvbnNdCiAgICBCIC0tPiBCMltNZW50b3IgSnVuaW9yIERldmVsb3BlcnNdCiAgICBCIC0tPiBCM1tJbXBsZW1lbnQgQ29tcGxleCBGZWF0dXJlc10KICAgIEIgLS0+IEI0W0NvZGUgUmV2aWV3XQogICAgQiAtLT4gQjVbUXVhbGl0eSBBc3N1cmFuY2VdCiAgICAKICAgIEMgLS0+IEMxW1NlY3VyaXR5IEF1ZGl0XQogICAgQyAtLT4gQzJbU3VwcG9ydCBCYWNrZW5kIERldmVsb3BtZW50XQogICAgQyAtLT4gQzNbTnV0cml0aW9uIExvZ2dpbmcgQVBJIEludGVncmF0aW9uXQogICAgCiAgICBDMSAtLT4gQzFhW0NvbmR1Y3QgU2VjdXJpdHkgQXVkaXRdCiAgICBDMSAtLT4gQzFiW0ltcGxlbWVudCBTZWN1cml0eSBJbXByb3ZlbWVudHNdCiAgICBDMSAtLT4gQzFjW1JldmlldyBBdXRoZW50aWNhdGlvbiBNZWNoYW5pc21zXQogICAgCiAgICBDMiAtLT4gQzJhW0Fzc2lzdCB3aXRoIEFQSSBEZXZlbG9wbWVudF0KICAgIEMyIC0tPiBDMmJbUmV2aWV3IEJhY2tlbmQgQXJjaGl0ZWN0dXJlXQogICAgCiAgICBDMyAtLT4gQzNhW1Jlc2VhcmNoIEZvb2QgRGF0YWJhc2UgQVBJc10KICAgIEMzIC0tPiBDM2J

In [84]:
backlog_json_path = '../../../data_/project1/backlog/sprint_planning_backlog.json'
upload_backlog_to_github(backlog_json_path)

Issue created successfully: Design and implement UI for activity logging form
Issue URL: https://github.com/VarjuAkos/HealthTrack-Pro/issues/3
Issue created successfully: Create API endpoint for saving activity data
Issue URL: https://github.com/VarjuAkos/HealthTrack-Pro/issues/4
Issue created successfully: Implement data validation for activity logging
Issue URL: https://github.com/VarjuAkos/HealthTrack-Pro/issues/5
Issue created successfully: Design and implement activity trends visualization
Issue URL: https://github.com/VarjuAkos/HealthTrack-Pro/issues/6
Issue created successfully: Design and implement overall dashboard layout
Issue URL: https://github.com/VarjuAkos/HealthTrack-Pro/issues/7
Issue created successfully: Create reusable card components for dashboard widgets
Issue URL: https://github.com/VarjuAkos/HealthTrack-Pro/issues/8
Issue created successfully: Implement API endpoints for health metrics CRUD operations
Issue URL: https://github.com/VarjuAkos/HealthTrack-Pro/issues